<a href="https://colab.research.google.com/github/naolia1211/machine_learning_Algorithms/blob/main/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O_on_Malware.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Before training

In [ ]:
!pip install h2o

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, roc_curve, auc
from sklearn import tree

import h2o
from h2o.estimators.glm import H2OGeneralizedLinearEstimator
from h2o.estimators.random_forest import H2ORandomForestEstimator
from h2o.estimators.gbm import H2OGradientBoostingEstimator
from h2o.grid.grid_search import H2OGridSearch
from h2o.estimators.stackedensemble import H2OStackedEnsembleEstimator


import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
#Initialize H2o
h2o.init()

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.17" 2022-10-18; OpenJDK Runtime Environment (build 11.0.17+8-post-Ubuntu-1ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.17+8-post-Ubuntu-1ubuntu218.04, mixed mode, sharing)
  Starting server from /usr/local/lib/python3.7/dist-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpe7d26lfg
  JVM stdout: /tmp/tmpe7d26lfg/h2o_unknownUser_started_from_python.out
  JVM stderr: /tmp/tmpe7d26lfg/h2o_unknownUser_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,02 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.38.0.3
H2O_cluster_version_age:,6 days
H2O_cluster_name:,H2O_from_python_unknownUser_eues2r
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,3.172 Gb
H2O_cluster_total_cores:,2
H2O_cluster_allowed_cores:,2
H2O_cluster_status:,"locked, healthy"


In [ ]:
# Reading dataset from Google drive
train = pd.read_csv('/content/drive/MyDrive/KDD/KDDTrain+.txt')
test = pd.read_csv('/content/drive/MyDrive/KDD/KDDTest+.txt')

In [ ]:
header = ['duration', 'protocol_type', 'service',
                          'src_bytes', 'dst_bytes', 'flag', 'land',
                          'wrong_fragment', 'urgent', 'hot', 'num_failed_logins',
                          'logged_in', 'num_compromised', 'root_shell',
                          'su_attempted', 'num_root', 'num_file_creations',
                          'num_shells', 'num_access_files', 'num_outbound_cmds',
                          'is_hot_login', 'is_guest_login', 'count', 'serror_rate',
                          'rerror_rate', 'same_srv_rate', 'diff_srv_rate', 'srv_count',
                          'srv_serror_rate', 'srv_rerror_rate', 'srv_diff_host_rate', 'dst_host_count',
                          'dst_host_srv_count', 'dst_host_same_srv_rate', 'dst_host_diff_srv_rate',
                          'dst_host_same_src_port_rate', 'dst_host_srv_diff_host_rate',
                          'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'dst_host_rerror_rate',
                          'dst_host_srv_rerror_rate', 'class', 'servere_points']

train.columns = header 
test.columns = header

In [ ]:
def dataPrep(data, label, drops):
  # We exclude all numeric columns
  num_cols = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
  
  noEncode = data.pop(label)

  data.drop(drops, axis = 1, inplace=True)

  #print(data)

  data_catcol = data.select_dtypes(exclude=num_cols)
  data_numcols = data.select_dtypes(include=num_cols)
  # Steps to one-hot encoding:
  # We iterate through each categorical column name
  # Create encoded variables for each categorical columns
  # Concatenate the encoded variables to the DataFrame
  # Remove the original categorical variable
  for col in data_catcol.columns.values:
    one_hot_encoded_variables = pd.get_dummies(data_catcol[col],prefix=col)
    data_catcol = pd.concat([data_catcol,one_hot_encoded_variables],axis=1)
    data_catcol.drop([col],axis=1, inplace=True)

  noEncode = pd.Series(map(lambda x: 1 if x == 'normal' else 0, noEncode), name='class')
  
  data = pd.concat([data_numcols, data_catcol, noEncode], axis=1)

  return data

train = dataPrep(train, 'class', ['servere_points'])
test = dataPrep(test, 'class', ['servere_points'])

In [ ]:
train = h2o.H2OFrame(train)
test = h2o.H2OFrame(test)

Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%
Parse progress: |████████████████████████████████████████████████████████████████| (done) 100%


In [ ]:
# Convert the categorical variables into factors

categorical_cols = ['protocol_type_icmp', 'protocol_type_tcp', 'protocol_type_udp',
 'service_IRC', 'service_X11', 'service_Z39_50', 'service_aol',
 'service_auth', 'service_bgp', 'service_courier', 'service_csnet_ns',
 'service_ctf', 'service_daytime', 'service_discard', 'service_domain',
 'service_domain_u', 'service_echo', 'service_eco_i', 'service_ecr_i',
 'service_efs', 'service_exec', 'service_finger', 'service_ftp',
 'service_ftp_data', 'service_gopher', 'service_harvest', 'service_hostnames',
 'service_http', 'service_http_2784', 'service_http_443', 'service_http_8001',
 'service_imap4', 'service_iso_tsap', 'service_klogin', 'service_kshell',
 'service_ldap', 'service_link', 'service_login', 'service_mtp',
 'service_name', 'service_netbios_dgm', 'service_netbios_ns', 'service_netbios_ssn',
 'service_netstat', 'service_nnsp', 'service_nntp', 'service_ntp_u',
 'service_other', 'service_pm_dump', 'service_pop_2', 'service_pop_3',
 'service_printer', 'service_private', 'service_red_i', 'service_remote_job',
 'service_rje', 'service_shell', 'service_smtp', 'service_sql_net', 'service_ssh',
 'service_sunrpc', 'service_supdup', 'service_systat', 'service_telnet',
 'service_tftp_u', 'service_tim_i', 'service_time', 'service_urh_i', 'service_urp_i',
 'service_uucp', 'service_uucp_path', 'service_vmnet', 'service_whois',
 'src_bytes_OTH', 'src_bytes_REJ', 'src_bytes_RSTO', 'src_bytes_RSTOS0',
 'src_bytes_RSTR', 'src_bytes_S0', 'src_bytes_S1', 'src_bytes_S2',
 'src_bytes_S3', 'src_bytes_SF', 'src_bytes_SH',]

for col in categorical_cols:
  train[col] = train[col].asfactor()

train['class'] = train['class'].asfactor()
train['class'].levels()



[['0', '1']]

In [ ]:
# Define predictors manually

predictors = [col for col in train.columns if col != 'class']

target = 'class'


In [ ]:
def report(trainedModel):
  print("AUC: ", trainedModel.auc())
  print("F1: ", trainedModel.F1())
  print("MSE: ", trainedModel.mse())
  print("Gini: ", trainedModel.gini())
  print("F0.5: ", trainedModel.F0point5())
  print("F2: ", trainedModel.F2())
  print("Accuracy: ", trainedModel.accuracy())
  print("Log loss: ", trainedModel.logloss())
  print("MSE: ", trainedModel.mse())
  print("Confusion matrix: ", trainedModel.confusion_matrix())

#GENERALIZED LINEAR MODEL (Defaut Settings)

STANDARDIZATION is enabled by default

GLM with default setting
GLM using lmbda search
GLM using Grid search
GLM WITH DEFAULT SETTINGS

Logistic Regression (Binomial Family)

H2O's GLM has the "family" argument, where the family is 'binomial' if the data is categorical 2 levels/classes or binary (Enum or Int).

In [ ]:
GLM_default_settings = H2OGeneralizedLinearEstimator(family='binomial', \
                                            model_id='GLM_default',nfolds = 10, \
                                            fold_assignment = "Modulo", \
                                            keep_cross_validation_predictions = True)

GLM_default_settings.train(x = predictors, y = target, training_frame = train)

glm Model Build progress: |

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:193: RuntimeWarning: Dropping bad and constant columns: [num_outbound_cmds]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_default


GLM Model: summary
    family    link    regularization                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ---------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    binomial  logit   Elastic Net (alpha = 0.5, lambda = 7.501E-4 )  205                           49                             7                       py_85_sid_b6d4

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.022708582746148242
RMSE: 0.15069367188488123
LogLoss: 0.08432532526718645
AUC: 0.9951001382010806
AUCPR: 0.9933914837924402
Gini: 0.9902002764021611
Null degrees of freedom: 125971
Residual degrees of freedom: 125922
Null deviance: 174031.28559199083
Residual deviance: 21245.25971254843
AIC: 21345.25971254843

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5902303599761325
       0      1      Error    Rate
-----  -----  -----  -------  -----------------
0      56588  2042   0.0348   (2042.0/58630.0)
1      1675   65667  0.0249   (1675.0/67342.0)
Total  58263  67709  0.0295   (3717.0/125972.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.59023      0.972477  196
max f2                       0.149632     0.984103  306
max f0point5                 0.706363     0.974404  164
max accuracy                 0.59023      0.970493  196
max precision                0.979894     0.998725  34
max recall                   5.04564e-05  1         399
max specificity              0.999872     0.999539  0
max absolute_mcc             0.59023      0.940695  196
max min_per_class_accuracy   0.64181      0.969589  182
max mean_per_class_accuracy  0.602873     0.970157  193
max tns                      0.999872     58603     0
max fns                      0.999872     66948     0
max fps                      5.04564e-05  58630     399
max tps                      5.04564e-05  67342     399
max tnr                      0.999872     0.999539  0
max fnr                      0.999872     0.994149  0
max fpr                      5.04564e-05  1         399
max tpr                      5.04564e-05  1         399

Gains/Lift Table: Avg response rate: 53.46 %, avg score: 53.46 %
group    cumulative_data_fraction    lower_threshold    lift         cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -----------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100022                   0.998176           1.82906      1.82906            0.977778         0.998955     0.977778                    0.998955            0.0182947       0.0182947                  82.9061   82.9061            0.0178171
2        0.0200044                   0.997542           1.87063      1.84985            1                0.997787     0.988889                    0.998371            0.0187105       0.0370051                  87.0631   84.9846            0.0365276
3        0.0300067                   0.997109           1.87063      1.85677            1                0.997309     0.992593                    0.998017            0.0187105       0.0557156                  87.0631   85.6774            0.055238
4        0.040001                    0.99691            1.87063      1.86024            1                0.997003     0.994443                    0.997764            0.0186956      

In [ ]:
print(h2o.save_model(model=GLM_default_settings, path="/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/", force=True))

/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM_default


In [ ]:
''' GLM_default_settings = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM_default')
print(h2o.get_model(GLM_default_settings)) '''

" GLM_default_settings = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM_default')\nprint(h2o.get_model(GLM_default_settings)) "

# **GLM WITH LAMBDA SEARCH**

The model parameter, lambda, controls the amount of regularization in a GLM model
Setting  lambda_search = True gives us optimal lambda value for the regularization strength.

In [ ]:
GLM_regularized = H2OGeneralizedLinearEstimator(family='binomial', model_id='GLM', \
                                                lambda_search=True, nfolds = 10, \
                                                fold_assignment = "Modulo", \
                                                keep_cross_validation_predictions = True)

GLM_regularized.train(x = predictors, y = target,training_frame = train)

glm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM


GLM Model: summary
    family    link    regularization                                 lambda_search                                                                     number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ---------------------------------------------  --------------------------------------------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    binomial  logit   Elastic Net (alpha = 0.5, lambda = 7.501E-5 )  nlambda = 100, lambda.max = 0.7501, lambda.min = 7.501E-5, lambda.1se = 1.088E-4  205                           75                             199                     py_85_sid_b6d4

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.020198451315634892
RMSE: 0.14212125567850467
LogLoss: 0.0740389742894486
AUC: 0.9962924570350009
AUCPR: 0.9950822908211506
Gini: 0.9925849140700018
Null degrees of freedom: 125971
Residual degrees of freedom: 125896
Null deviance: 174031.28559199083
Residual deviance: 18665.974327418902
AIC: 18817.974327418902

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6545620391268526
       0      1      Error    Rate
-----  -----  -----  -------  -----------------
0      57133  1497   0.0255   (1497.0/58630.0)
1      1790   65552  0.0266   (1790.0/67342.0)
Total  58923  67049  0.0261   (3287.0/125972.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.654562     0.975542  182
max f2                       0.14696      0.98461   317
max f0point5                 0.85867      0.978954  115
max accuracy                 0.654562     0.973907  182
max precision                0.986369     0.998229  30
max recall                   2.34634e-05  1         399
max specificity              0.999688     0.999505  0
max absolute_mcc             0.654562     0.94759   182
max min_per_class_accuracy   0.646428     0.973785  184
max mean_per_class_accuracy  0.667932     0.973946  179
max tns                      0.999688     58601     0
max fns                      0.999688     63754     0
max fps                      2.34634e-05  58630     399
max tps                      2.34634e-05  67342     399
max tnr                      0.999688     0.999505  0
max fnr                      0.999688     0.94672   0
max fpr                      2.34634e-05  1         399
max tpr                      2.34634e-05  1         399

Gains/Lift Table: Avg response rate: 53.46 %, avg score: 53.46 %
group    cumulative_data_fraction    lower_threshold    lift         cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  -----------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100022                   0.999702           1.82906      1.82906            0.977778         0.999852     0.977778                    0.999852            0.0182947       0.0182947                  82.9061   82.9061            0.0178171
2        0.0200044                   0.999626           1.87063      1.84985            1                0.999667     0.988889                    0.999759            0.0187105       0.0370051                  87.0631   84.9846            0.0365276
3        0.0300067                   0.999439           1.87063      1.85677            1                0.999543     0.992593                    0.999687            0.0187105       0.05571

In [ ]:
GLM_regularized.confusion_matrix

<bound method H2OBinomialModel.confusion_matrix of H2OGeneralizedLinearEstimator({'parms': {'model_id': {'__meta': {'schema_version': 3, 'schema_name': 'ModelParameterSchemaV3', 'schema_type': 'Iced'}, 'name': 'model_id', 'label': 'model_id', 'help': 'Destination id for this model; auto-generated if not specified.', 'required': False, 'type': 'Key<Model>', 'default_value': None, 'actual_value': {'__meta': {'schema_version': 3, 'schema_name': 'ModelKeyV3', 'schema_type': 'Key<Model>'}, 'name': 'GLM', 'type': 'Key<Model>', 'URL': '/3/Models/GLM'}, 'input_value': None, 'level': 'critical', 'values': [], 'is_member_of_frames': [], 'is_mutually_exclusive_with': [], 'gridable': False}, 'training_frame': {'__meta': {'schema_version': 3, 'schema_name': 'ModelParameterSchemaV3', 'schema_type': 'Iced'}, 'name': 'training_frame', 'label': 'training_frame', 'help': 'Id of the training data frame.', 'required': False, 'type': 'Key<Frame>', 'default_value': None, 'actual_value': {'__meta': {'schema_

In [ ]:
print(h2o.save_model(model=GLM_regularized, path="/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/", force=True))

/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM


In [ ]:
''' GLM_regularized = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM')
print(h2o.get_model(GLM_regularized)) '''

" GLM_regularized = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM')\nprint(h2o.get_model(GLM_regularized)) "

#**GLM WITH GRID SEARCH**

GLM needs to find the optimal values of the regularization parameters α and λ
lambda: controls the amount of regularization, when set to 0 it gets disabled

alpha : controls the distribution between lasso & ridge regression penalties.

random grid search: H2o supports 2 types of grid search, cartesian and random. We make use of the random as the search criteria for faster computation

Stopping metric: we specify the metric used for early stopping. AUTO takes log loss as default

source: http://docs.h2o.ai/h2o/latest-stable/h2o-docs/data-science/algo-params/lambda.html



In [ ]:
hyper_parameters = { 'alpha': [0.0001, 0.001, 0.01, 0.1],
                     'lambda': [0.001, 0.01, 0.1] }
search_criteria = { 'strategy': "RandomDiscrete", 
                    'stopping_metric': "AUTO",
                    'stopping_rounds': 5}

GLM_grid_search = H2OGridSearch(H2OGeneralizedLinearEstimator(family='binomial', \
                  nfolds = 10, fold_assignment = "Modulo", \
                  keep_cross_validation_predictions = True),\
                  hyper_parameters, grid_id="GLM_grid", search_criteria=search_criteria)

GLM_grid_search.train(x= predictors,y= target, training_frame=train)


glm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


/usr/local/lib/python3.7/dist-packages/h2o/grid/grid_search.py:431: UserWarning: Adding alpha array to hyperparameter runs slower with gridsearch. This is due to the fact that the algo has to run initialization for every alpha value. Setting the alpha array as a model parameter will skip the initialization and run faster overall.
  warnings.warn(w_message)


,alpha,lambda,model_ids,logloss
,0.0001,0.001,GLM_grid_model_6,0.0865000
,0.001,0.001,GLM_grid_model_8,0.0865215
,0.01,0.001,GLM_grid_model_12,0.0865592
,0.1,0.001,GLM_grid_model_7,0.0869008
,0.0001,0.01,GLM_grid_model_1,0.1159810
,0.001,0.01,GLM_grid_model_4,0.1160181
,0.01,0.01,GLM_grid_model_3,0.1162848
,0.1,0.01,GLM_grid_model_9,0.1202857
,0.0001,0.1,GLM_grid_model_2,0.1826163
,0.001,0.1,GLM_grid_model_10,0.1828146


# Get the grid results, sorted by validation AUC
  

In [ ]:
# Get the grid results, sorted by validation AUC
GLM_grid_sorted = GLM_grid_search.get_grid(sort_by='auc', decreasing=True)
GLM_grid_sorted

,alpha,lambda,model_ids,auc
,0.0001,0.001,GLM_grid_model_6,0.9947265
,0.001,0.001,GLM_grid_model_8,0.9947118
,0.01,0.001,GLM_grid_model_12,0.9947036
,0.1,0.001,GLM_grid_model_7,0.9946563
,0.0001,0.01,GLM_grid_model_1,0.9907448
,0.001,0.01,GLM_grid_model_4,0.9907383
,0.01,0.01,GLM_grid_model_3,0.9907224
,0.1,0.01,GLM_grid_model_9,0.9903759
,0.0001,0.1,GLM_grid_model_2,0.9859579
,0.001,0.1,GLM_grid_model_10,0.9859377


In [ ]:
# Extract the best model from random grid search
Best_GLM_model_from_Grid = GLM_grid_sorted.model_ids[0]

#model performance
Best_GLM_model_from_Grid = h2o.get_model(Best_GLM_model_from_Grid)
print(Best_GLM_model_from_Grid)

Model Details
H2OGeneralizedLinearEstimator : Generalized Linear Modeling
Model Key: GLM_grid_model_6


GLM Model: summary
    family    link    regularization                                 number_of_predictors_total    number_of_active_predictors    number_of_iterations    training_frame
--  --------  ------  ---------------------------------------------  ----------------------------  -----------------------------  ----------------------  ----------------
    binomial  logit   Elastic Net (alpha = 1.0E-4, lambda = 0.001 )  205                           123                            7                       py_85_sid_b6d4

ModelMetricsBinomialGLM: glm
** Reported on train data. **

MSE: 0.022744166710172143
RMSE: 0.15081169288278726
LogLoss: 0.08527331372651978
AUC: 0.9949105146141968
AUCPR: 0.9932161188607552
Gini: 0.9898210292283935
Null degrees of freedom: 125971
Residual degrees of freedom: 125848
Null deviance: 174031.28559199083
Residual deviance: 21484.099753551214
AIC: 21732.

In [ ]:
print(h2o.save_model(model=Best_GLM_model_from_Grid, path="/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/", force=True))

/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM_grid_model_6


In [ ]:
''' Best_GLM_model_from_Grid = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM_grid_model_8')
print(h2o.get_model(Best_GLM_model_from_Grid)) '''

" Best_GLM_model_from_Grid = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM_grid_model_8')\nprint(h2o.get_model(Best_GLM_model_from_Grid)) "

# RF WITH DEFAULT SETTINGS

  

In [ ]:
# Build a RF model with default settings
RF_default_settings = H2ORandomForestEstimator(model_id = 'RF_D',\
                                nfolds = 10, fold_assignment = "Modulo", \
                                keep_cross_validation_predictions = True)

# Use train() to build the model
RF_default_settings.train(x = predictors, y = target, training_frame = train)

drf Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2ORandomForestEstimator : Distributed Random Forest
Model Key: RF_D


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          259939                 20           20           20            256           509           406

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.003955405617020052
RMSE: 0.06289201552677455
LogLoss: 0.021744973819414745
Mean Per-Class Error: 0.0034146831805814603
AUC: 0.9999168907623459
AUCPR: 0.999925593692654
Gini: 0.9998337815246918

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.4570281875080659
       0      1      Error    Rate
-----  -----  -----  -------  ----------------
0      58348  282    0.0048   (282.0/58630.0)
1      136    67206  0.002    (136.0/67342.0)
Total  58484  67488  0.0033   (418.0/125972.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.457028     0.9969    216
max f2                       0.388591     0.997973  231
max f0point5                 0.69188      0.997075  165
max accuracy                 0.457028     0.996682  216
max precision                0.99983      1         0
max recall                   0.0630428    1         346
max specificity              0.99983      1         0
max absolute_mcc             0.457028     0.993333  216
max min_per_class_accuracy   0.532751     0.996496  201
max mean_per_class_accuracy  0.479228     0.996607  212
max tns                      0.99983      58630     0
max fns                      0.99983      62966     0
max fps                      3.23255e-05  58630     399
max tps                      0.0630428    67342     346
max tnr                      0.99983      1         0
max fnr                      0.99983      0.935018  0
max fpr                      3.23255e-05  1         399
max tpr                      0.0630428    1         346

Gains/Lift Table: Avg response rate: 53.46 %, avg score: 53.45 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0100022                   0.999914           1.87063   1.87063            1                0.999961     1                           0.999961            0.0187105       0.0187105                  87.0631   87.0631            0.0187105
2        0.0200044                   0.999824           1.87063   1.87063            1                0.99987      1                           0.999915            0.0187105       0.0374209                  87.0631   87.0631            0.0374209
3        0.0300067                   0.999691           1.87063   1.87063            1                0.999765     1                           0.999865            0.0187105       0.0561314                  87.0631   87.0631            0.0561314
4        0.040001                    0.999515           1.87063   1.87063            1                0.999605     1                           0.9998              0.0186956       0.074827                   87.0631   87.0631            0.074827
5        0.0500032                   0.999358           1.87063   1.87063            1                0.999437     1                           0.99

In [ ]:
#Let's see the default parameters that RF model utilizes:
RF_default_settings.summary()

,number_of_trees,number_of_internal_trees,model_size_in_bytes,min_depth,max_depth,mean_depth,min_leaves,max_leaves,mean_leaves
,50.0,50.0,259939.0,20.0,20.0,20.0,256.0,509.0,406.0


In [ ]:
print(h2o.save_model(model=RF_default_settings, path="/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/", force=True))

/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/RF_D


In [ ]:
''' RF_default_settings = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/RF_D') 
print(h2o.get_model(RF_default_settings)'''

" RF_default_settings = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/RF_D') \nprint(h2o.get_model(RF_default_settings)"

# RF with GRID SEARCH to extract the best model

  

In [ ]:
hyper_params = {'sample_rate':[0.7,0.9],
                'col_sample_rate_per_tree': [0.8, 0.9],
                'max_depth': [3, 5],
                'ntrees': [200, 300]
               }

In [ ]:
RF_grid_search = H2OGridSearch(H2ORandomForestEstimator(nfolds = 10, \
                             fold_assignment = "Modulo", \
                             keep_cross_validation_predictions = True, \
                             stopping_metric = 'AUC',stopping_rounds = 5), \
                             hyper_params = hyper_params, \
                             grid_id= 'RF_gridsearch')

# Use train() to start the grid search
RF_grid_search.train(x = predictors, y = target, training_frame = train)

drf Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,col_sample_rate_per_tree,max_depth,ntrees,sample_rate,model_ids,logloss
,0.9,5.0,83.0,0.7,RF_gridsearch_model_8,0.1195883
,0.8,5.0,9.0,0.7,RF_gridsearch_model_3,0.1207654
,0.9,5.0,21.0,0.7,RF_gridsearch_model_4,0.1207921
,0.8,5.0,10.0,0.9,RF_gridsearch_model_11,0.1227574
,0.9,5.0,49.0,0.9,RF_gridsearch_model_12,0.1257021
,0.9,5.0,19.0,0.9,RF_gridsearch_model_16,0.1281171
,0.8,5.0,15.0,0.7,RF_gridsearch_model_7,0.1283488
,0.8,5.0,39.0,0.9,RF_gridsearch_model_15,0.1296725
,0.8,3.0,14.0,0.7,RF_gridsearch_model_1,0.1805834
,0.9,3.0,7.0,0.7,RF_gridsearch_model_6,0.1888521


In [ ]:
# Sort the grid models
RF_grid_sorted = RF_grid_search.get_grid(sort_by='auc', decreasing=True)
print(RF_grid_sorted)

Hyper-Parameter Search Summary: ordered by decreasing auc
    col_sample_rate_per_tree    max_depth    ntrees    sample_rate    model_ids               auc
--  --------------------------  -----------  --------  -------------  ----------------------  --------
    0.9                         5            83        0.7            RF_gridsearch_model_8   0.99609
    0.8                         5            9         0.7            RF_gridsearch_model_3   0.995738
    0.9                         5            21        0.7            RF_gridsearch_model_4   0.99548
    0.8                         5            15        0.7            RF_gridsearch_model_7   0.995327
    0.9                         5            49        0.9            RF_gridsearch_model_12  0.994963
    0.9                         5            19        0.9            RF_gridsearch_model_16  0.994386
    0.8                         5            39        0.9            RF_gridsearch_model_15  0.994331
    0.8               

In [ ]:
# Extract the best model from random grid search
Best_RF_model_from_Grid = RF_grid_sorted.model_ids[0]

# Model performance
Best_RF_model_from_Grid = h2o.get_model(Best_RF_model_from_Grid) 
print(Best_RF_model_from_Grid)

Model Details
H2ORandomForestEstimator : Distributed Random Forest
Model Key: RF_gridsearch_model_8


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    83                 83                          30482                  5            5            5             14            32            24.494

ModelMetricsBinomial: drf
** Reported on train data. **

MSE: 0.02986948777216001
RMSE: 0.17282791375284262
LogLoss: 0.12265954440935405
Mean Per-Class Error: 0.026968617473474007
AUC: 0.9949837037894649
AUCPR: 0.9951737574167863
Gini: 0.9899674075789298

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.6343285948222775
       0      1      Error    Rate
-----  -----  -----  -------  -----------------
0      5617

In [ ]:
print(h2o.save_model(model=Best_RF_model_from_Grid, path="/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/", force=True))

/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/RF_gridsearch_model_8


In [ ]:
''' Best_RF_model_from_Grid = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/RF_gridsearch_model_23')
print(h2o.get_model(Best_RF_model_from_Grid)) '''

" Best_RF_model_from_Grid = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/RF_gridsearch_model_23')\nprint(h2o.get_model(Best_RF_model_from_Grid)) "

#GBM WITH DEFAULT SETTINGS

In [ ]:
GBM_default_settings = H2OGradientBoostingEstimator(model_id = 'GBM_default', \
                       nfolds = 10, \
                       fold_assignment = "Modulo", \
                       keep_cross_validation_predictions = True)

# Use train() to build the model
GBM_default_settings.train(x = predictors, y = target, training_frame = train)

gbm Model Build progress: |██████████████████████████████████████████████████████| (done) 100%


Model Details
=============
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_default


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    50                 50                          21596                  5            5            5             18            32            29.68

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.00361672524168912
RMSE: 0.060139215506099845
LogLoss: 0.02263369533905072
Mean Per-Class Error: 0.003802084322956667
AUC: 0.999777934412682
AUCPR: 0.9997948901414289
Gini: 0.9995558688253641

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5824246515713167
       0      1      Error    Rate
-----  -----  -----  -------  ----------------
0      58367  263    0.0045   (263.0/58630.0)
1      210    67132  0.0031   (210.0/67342.0)
Total  58577  67395  0.0038   (473.0/125972.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.582425     0.996489  195
max f2                       0.345129     0.997425  241
max f0point5                 0.673215     0.996571  176
max accuracy                 0.586253     0.996245  194
max precision                0.994519     1         0
max recall                   0.021619     1         369
max specificity              0.994519     1         0
max absolute_mcc             0.582425     0.992454  195
max min_per_class_accuracy   0.643837     0.996009  184
max mean_per_class_accuracy  0.586253     0.9962    194
max tns                      0.994519     58630     0
max fns                      0.994519     60753     0
max fps                      0.00511372   58630     399
max tps                      0.021619     67342     369
max tnr                      0.994519     1         0
max fnr                      0.994519     0.902156  0
max fpr                      0.00511372   1         399
max tpr                      0.021619     1         369

Gains/Lift Table: Avg response rate: 53.46 %, avg score: 53.46 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score       cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  ----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.0519242                   0.99452            1.87063   1.87063            1                0.994521    1                           0.994521            0.0971311       0.0971311                  87.0631   87.0631            0.0971311
2        0.283595                    0.994451           1.87063   1.87063            1                0.994451    1                           0.994464            0.43337         0.530501                   87.0631   87.0631            0.530501
3        0.306584                    0.993843           1.87063   1.87063            1                0.994125    1                           0.994438            0.0430044       0.573505                   87.0631   87.0631            0.573505
4        0.400264                    0.987849           1.87      1.87048            0.999661         0.992073    0.999921                    0.993885            0.17518         0.748686                   86.9996   87.0482            0.748618
5        0.5                         0.959851           1.8608    1.86855            0.994747         0.977488    0.998889                    0.

In [ ]:
print(h2o.save_model(model=GBM_default_settings, path="/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/", force=True))

/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GBM_default


In [ ]:
''' GBM_default_settings = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GBM_default')
print(h2o.get_model(GBM_default_settings)) '''

" GBM_default_settings = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GBM_default')\nprint(h2o.get_model(GBM_default_settings)) "

#GBM WITH GRID SEARCH

In [ ]:
hyper_params = {'sample_rate': [0.8, 0.9],
                'col_sample_rate': [0.2, 0.5],
                'max_depth': [5, 9],
                'ntrees' : [100, 200]
               }

In [ ]:
GBM_grid_search = H2OGridSearch(H2OGradientBoostingEstimator(nfolds = 10, \
                        fold_assignment = "Modulo", \
                        keep_cross_validation_predictions = True,\
                        stopping_metric = 'AUC', stopping_rounds = 5),
                        hyper_params = hyper_params, grid_id= 'GBM_Grid')

# Use train() to start the grid search
GBM_grid_search.train(x = predictors, y = target, training_frame = train)

gbm Grid Build progress: |███████████████████████████████████████████████████████| (done) 100%


,col_sample_rate,max_depth,ntrees,sample_rate,model_ids,logloss
,0.2,9.0,84.0,0.8,GBM_Grid_model_7,0.0075273
,0.2,9.0,69.0,0.9,GBM_Grid_model_15,0.0089321
,0.2,9.0,64.0,0.8,GBM_Grid_model_3,0.0095565
,0.2,5.0,196.0,0.8,GBM_Grid_model_5,0.0102657
,0.2,5.0,187.0,0.9,GBM_Grid_model_13,0.0110438
,0.5,5.0,116.0,0.8,GBM_Grid_model_6,0.0112247
,0.2,9.0,54.0,0.9,GBM_Grid_model_11,0.0112784
,0.5,9.0,50.0,0.9,GBM_Grid_model_16,0.0115519
,0.5,5.0,116.0,0.9,GBM_Grid_model_14,0.0116728
,0.5,5.0,99.0,0.9,GBM_Grid_model_10,0.0126291


In [ ]:
# Sort and show the grid search results
GBM_grid_sorted = GBM_grid_search.get_grid(sort_by='auc', decreasing=True)
print(GBM_grid_sorted)

Hyper-Parameter Search Summary: ordered by decreasing auc
    col_sample_rate    max_depth    ntrees    sample_rate    model_ids          auc
--  -----------------  -----------  --------  -------------  -----------------  --------
    0.2                9            84        0.8            GBM_Grid_model_7   0.999962
    0.2                9            64        0.8            GBM_Grid_model_3   0.999952
    0.2                9            69        0.9            GBM_Grid_model_15  0.999951
    0.5                9            50        0.9            GBM_Grid_model_16  0.999945
    0.2                9            54        0.9            GBM_Grid_model_11  0.999944
    0.5                9            46        0.8            GBM_Grid_model_4   0.999943
    0.5                9            47        0.9            GBM_Grid_model_12  0.99993
    0.5                9            45        0.8            GBM_Grid_model_8   0.999926
    0.2                5            196       0.8         

In [ ]:
# Extract the best model from random grid search
Best_GBM_model_from_Grid = GBM_grid_sorted.model_ids[0]

Best_GBM_model_from_Grid = h2o.get_model(Best_GBM_model_from_Grid) 
print(Best_GBM_model_from_Grid)

Model Details
H2OGradientBoostingEstimator : Gradient Boosting Machine
Model Key: GBM_Grid_model_7


Model Summary: 
    number_of_trees    number_of_internal_trees    model_size_in_bytes    min_depth    max_depth    mean_depth    min_leaves    max_leaves    mean_leaves
--  -----------------  --------------------------  ---------------------  -----------  -----------  ------------  ------------  ------------  -------------
    84                 84                          167241                 9            9            9             35            262           153.738

ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 0.0009911023909017806
RMSE: 0.03148177871248352
LogLoss: 0.005508074004352803
Mean Per-Class Error: 0.0010955280048753407
AUC: 0.9999847792501563
AUCPR: 0.9999864748326862
Gini: 0.9999695585003125

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.5730558786944447
       0      1      Error    Rate
-----  -----  -----  -------  ----------------
0      5

In [ ]:
print(h2o.save_model(model=Best_GBM_model_from_Grid, path="/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/", force=True))

/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GBM_Grid_model_7


In [ ]:
''' Best_GBM_model_from_Grid = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GBM_Grid_model_17')
print(h2o.get_model(Best_GBM_model_from_Grid)) '''

" Best_GBM_model_from_Grid = h2o.load_model('/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GBM_Grid_model_17')\nprint(h2o.get_model(Best_GBM_model_from_Grid)) "

#STACKED ENSEMBLE

In [ ]:
# list the best models from each grid
all_models = [Best_GLM_model_from_Grid, Best_RF_model_from_Grid, Best_GBM_model_from_Grid]

In [ ]:
# Set up Stacked Ensemble
ensemble = H2OStackedEnsembleEstimator(model_id = "ensemble", base_models = all_models, metalearner_algorithm = "deeplearning")

# uses GLM as the default metalearner
ensemble.train(y = target, training_frame = train)

stackedensemble Model Build progress: |

/usr/local/lib/python3.7/dist-packages/h2o/estimators/estimator_base.py:193: RuntimeWarning: Dropping unused columns: [num_outbound_cmds]
  warnings.warn(mesg["message"], RuntimeWarning)


██████████████████████████████████████████| (done) 100%


Model Details
=============
H2OStackedEnsembleEstimator : Stacked Ensemble
Model Key: ensemble

No summary for this model

ModelMetricsBinomial: stackedensemble
** Reported on train data. **

MSE: 0.0006902073532732344
RMSE: 0.026271797678751152
LogLoss: 0.003318381435387688
Mean Per-Class Error: 0.0005390254420008624
AUC: 0.9999989936044773
AUCPR: 0.9999991286435705
Gini: 0.9999979872089546

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.00012455813756513583
       0     1     Error    Rate
-----  ----  ----  -------  ------------
0      4633  5     0.0011   (5.0/4638.0)
1      0     5356  0        (0.0/5356.0)
Total  4633  5361  0.0005   (5.0/9994.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.000124558  0.999533  389
max f2                       0.000124558  0.999813  389
max f0point5                 0.968105     0.999663  375
max accuracy                 0.000124558  0.9995    389
max precision                0.999999     1         0
max recall                   0.000124558  1         389
max specificity              0.999999     1         0
max absolute_mcc             0.000124558  0.998995  389
max min_per_class_accuracy   0.107455     0.999353  384
max mean_per_class_accuracy  0.000124558  0.999461  389
max tns                      0.999999     4638      0
max fns                      0.999999     5350      0
max fps                      1.71514e-10  4638      399
max tps                      0.000124558  5356      389
max tnr                      0.999999     1         0
max fnr                      0.999999     0.99888   0
max fpr                      1.71514e-10  1         399
max tpr                      0.000124558  1         389

Gains/Lift Table: Avg response rate: 53.59 %, avg score: 53.53 %
group    cumulative_data_fraction    lower_threshold    lift      cumulative_lift    response_rate    score        cumulative_response_rate    cumulative_score    capture_rate    cumulative_capture_rate    gain      cumulative_gain    kolmogorov_smirnov
-------  --------------------------  -----------------  --------  -----------------  ---------------  -----------  --------------------------  ------------------  --------------  -------------------------  --------  -----------------  --------------------
1        0.010006                    0.999993           1.86594   1.86594            1                0.999997     1                           0.999997            0.0186706       0.0186706                  86.5945   86.5945            0.0186706
2        0.020012                    0.999993           1.86594   1.86594            1                0.999993     1                           0.999995            0.0186706       0.0373413                  86.5945   86.5945            0.0373413
3        0.030018                    0.999993           1.86594   1.86594            1                0.999993     1                           0.999994            0.0186706       0.0560119                  86.5945   86.5945            0.0560119
4        0.040024                    0.999993           1.86594   1.86594            1                0.999993     1                           0.999994            0.0186706       0.0746826                  86.5945   86.5945            0.0746826
5        0.05003                     0.999993           1.86594   1.86594            1                0.999993     1                           0.999994            0.0186706       0.0933532                  86.5945   86.5945            0.0933532
6        0.10006                     0.999992           1.86594   1.86594            1                0.999992     1                           0.999993            0.0933532       0.186706                   86.5945   86.5945            0.186706
7        0.14999                     0.999992           1.86594   1.86594            1                0.999992 

In [ ]:
print(h2o.save_model(model=ensemble, path="/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/", force=True))

/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/ensemble


### Checking model performance of all base learners

In [ ]:
# Checking the model performance for all GLM models built

model_perf_GLM_default = GLM_default_settings.model_performance(test)
print("---------------------------------------------------------------------------------")
print("Report for GLM default: ")
report(model_perf_GLM_default)
model_perf_GLM_regularized = GLM_regularized.model_performance(test)
print("---------------------------------------------------------------------------------")
print("Report for GLM_regularized: ")
report(model_perf_GLM_regularized)
model_perf_Best_GLM_model_from_Grid = Best_GLM_model_from_Grid.model_performance(test)
print("---------------------------------------------------------------------------------")
print("Report for Best_GLM_model_from_Grid: ")
report(model_perf_Best_GLM_model_from_Grid)

---------------------------------------------------------------------------------
Report for GLM default: 
AUC:  0.8715208442311994
F1:  [[0.9437380916455735, 0.8001539127507095]]
MSE:  0.22405602850221104
Gini:  0.7430416884623987
F0.5:  [[0.9818581846958288, 0.8149315342107015]]
F2:  [[0.8098619426352783, 0.8576628204135368]]
Accuracy:  [[0.9784644501851787, 0.8309896641973118]]
Log loss:  1.0154901591103882
MSE:  0.22405602850221104
Confusion matrix:  Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9437380916455735
       0      1      Error    Rate
-----  -----  -----  -------  ----------------
0      10070  2762   0.2152   (2762.0/12832.0)
1      1393   8318   0.1434   (1393.0/9711.0)
Total  11463  11080  0.1843   (4155.0/22543.0)
---------------------------------------------------------------------------------
Report for GLM_regularized: 
AUC:  0.829018721314056
F1:  [[0.96655908218939, 0.7967228915662651]]
MSE:  0.22396611285362147
Gini:  0.6580374426281119
F0.5:  [[0.971

In [ ]:
# Checking the model performance for all RF models built

model_perf_RF_default_settings = RF_default_settings.model_performance(test)
print("---------------------------------------------------------------------------------")
print("Report for RF_default_settings: ")
report(model_perf_RF_default_settings)

model_perf_Best_RF_model_from_Grid = Best_RF_model_from_Grid.model_performance(test)
print("---------------------------------------------------------------------------------")
print("Report for Best_RF_model_from_Grid: ")
report(model_perf_Best_RF_model_from_Grid)

---------------------------------------------------------------------------------
Report for RF_default_settings: 
AUC:  0.9542264508510414
F1:  [[0.9695379968309378, 0.8838955327934788]]
MSE:  0.16988970337777626
Gini:  0.9084529017020828
F0.5:  [[0.9859537609121832, 0.9244168398440237]]
F2:  [[0.5203240258799182, 0.8853069515498043]]
Accuracy:  [[0.9804022713873891, 0.9062680211152021]]
Log loss:  0.5468070427132175
MSE:  0.16988970337777626
Confusion matrix:  Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9695379968309378
       0      1     Error    Rate
-----  -----  ----  -------  ----------------
0      12137  695   0.0542   (695.0/12832.0)
1      1470   8241  0.1514   (1470.0/9711.0)
Total  13607  8936  0.096    (2165.0/22543.0)
---------------------------------------------------------------------------------
Report for Best_RF_model_from_Grid: 
AUC:  0.9397352101031532
F1:  [[0.9073016434399597, 0.8459818933720801]]
MSE:  0.1758784265475983
Gini:  0.8794704202063064
F0

In [ ]:
# Checking the model performance for all GBM models built

model_perf_GBM_default_settings = GBM_default_settings.model_performance(test)
print("---------------------------------------------------------------------------------")
print("Report for GBM_default_settings: ")
report(model_perf_GBM_default_settings)

model_perf_Best_GBM_model_from_Grid = Best_GBM_model_from_Grid.model_performance(test)
print("---------------------------------------------------------------------------------")
print("Report for Best_GBM_model_from_Grid: ")
report(model_perf_Best_GBM_model_from_Grid)

---------------------------------------------------------------------------------
Report for GBM_default_settings: 
AUC:  0.9430293228351734
F1:  [[0.9877113326646165, 0.8563647257475898]]
MSE:  0.17913870719862002
Gini:  0.8860586456703468
F0.5:  [[0.9927021709407542, 0.9210626474842047]]
F2:  [[0.16871014326029604, 0.8846275989613495]]
Accuracy:  [[0.9913261787428174, 0.8851971787251032]]
Log loss:  0.6707171055651723
MSE:  0.17913870719862002
Confusion matrix:  Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9877113326646165
       0      1     Error    Rate
-----  -----  ----  -------  ----------------
0      12045  787   0.0613   (787.0/12832.0)
1      1850   7861  0.1905   (1850.0/9711.0)
Total  13895  8648  0.117    (2637.0/22543.0)
---------------------------------------------------------------------------------
Report for Best_GBM_model_from_Grid: 
AUC:  0.9501784192528153
F1:  [[0.9923519309572126, 0.8677388260362233]]
MSE:  0.1893199152622362
Gini:  0.9003568385056306

### Best AUC from the base learners

In [ ]:
# Best AUC from the base learner models
best_auc = max(model_perf_GLM_default.auc(), model_perf_GLM_regularized.auc(), \
               model_perf_Best_GLM_model_from_Grid.auc(), \
               model_perf_RF_default_settings.auc(), \
               model_perf_Best_RF_model_from_Grid.auc(), \
               model_perf_GBM_default_settings.auc(), \
               model_perf_Best_GBM_model_from_Grid.auc())

print("Best AUC out of all the models performed: ", format(best_auc))

Best AUC out of all the models performed:  0.9542264508510414


### AUC from the Ensemble Learner

In [ ]:
# Eval ensemble performance on the test data
Ensemble_model = ensemble.model_performance(test)
print("---------------------------------------------------------------------------------")
print("Report for ensemble: ")
report(Ensemble_model)

---------------------------------------------------------------------------------
Report for ensemble: 
AUC:  0.932440007648729
F1:  [[0.9999910155305758, 0.8765419082762257]]
MSE:  0.1881140496450202
Gini:  0.8648800152974581
F0.5:  [[0.9999910155305758, 0.8892549254497615]]
F2:  [[0.9999340025481704, 0.9139721616899307]]
Accuracy:  [[0.9999910155305758, 0.8961096570997649]]
Log loss:  1.868792661393939
MSE:  0.1881140496450202
Confusion matrix:  Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999910155305758
       0      1     Error    Rate
-----  -----  ----  -------  ----------------
0      11887  945   0.0736   (945.0/12832.0)
1      1397   8314  0.1439   (1397.0/9711.0)
Total  13284  9259  0.1039   (2342.0/22543.0)


In [ ]:
print(Ensemble_model)

ModelMetricsBinomial: stackedensemble
** Reported on test data. **

MSE: 0.1881140496450202
RMSE: 0.43372116577937514
LogLoss: 1.868792661393939
Mean Per-Class Error: 0.10875074808473616
AUC: 0.932440007648729
AUCPR: 0.8797636626772504
Gini: 0.8648800152974581

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9999910155305758
       0      1     Error    Rate
-----  -----  ----  -------  ----------------
0      11887  945   0.0736   (945.0/12832.0)
1      1397   8314  0.1439   (1397.0/9711.0)
Total  13284  9259  0.1039   (2342.0/22543.0)

Maximum Metrics: Maximum metrics at their respective thresholds
metric                       threshold    value     idx
---------------------------  -----------  --------  -----
max f1                       0.999991     0.876542  0
max f2                       0.999934     0.913972  2
max f0point5                 0.999991     0.889255  0
max accuracy                 0.999991     0.89611   0
max precision                0.999991     0.897937  0
m

#Loading pretrained models

In [ ]:
''' # Set up Stacked Ensemble
ensemble = H2OStackedEnsembleEstimator(model_id = "ensemble", base_models = all_models, metalearner_algorithm = "deeplearning")

# uses GLM as the default metalearner
ensemble.train(y = target, training_frame = train)
ensemble = h2o.load_model("/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GBM_default")
Ensemble_model = ensemble.model_performance(test) '''

' # Set up Stacked Ensemble\nensemble = H2OStackedEnsembleEstimator(model_id = "ensemble", base_models = all_models, metalearner_algorithm = "deeplearning")\n\n# uses GLM as the default metalearner\nensemble.train(y = target, training_frame = train)\nensemble = h2o.load_model("/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GBM_default")\nEnsemble_model = ensemble.model_performance(test) '

In [ ]:
''' hyper_parameters = { 'alpha': [0.0001, 0.001, 0.01, 0.1],
                     'lambda': [0.001, 0.01, 0.1] }
search_criteria = { 'strategy': "RandomDiscrete", 
                    'stopping_metric': "AUTO",
                    'stopping_rounds': 5}

GLM_grid_search = H2OGridSearch(H2OGeneralizedLinearEstimator(family='binomial', \
                  nfolds = 10, fold_assignment = "Modulo", \
                  keep_cross_validation_predictions = True),\
                  hyper_parameters, grid_id="GLM_grid", search_criteria=search_criteria)

#GLM_grid_search.train(x= predictors,y= target, training_frame=train)
GLM_grid_search = h2o.load_model("/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM_grid_model_10")
GLM_grid_search_performance = GLM_grid_search.model_performance(test)
 '''

' hyper_parameters = { \'alpha\': [0.0001, 0.001, 0.01, 0.1],\n                     \'lambda\': [0.001, 0.01, 0.1] }\nsearch_criteria = { \'strategy\': "RandomDiscrete", \n                    \'stopping_metric\': "AUTO",\n                    \'stopping_rounds\': 5}\n\nGLM_grid_search = H2OGridSearch(H2OGeneralizedLinearEstimator(family=\'binomial\',                   nfolds = 10, fold_assignment = "Modulo",                   keep_cross_validation_predictions = True),                  hyper_parameters, grid_id="GLM_grid", search_criteria=search_criteria)\n\n#GLM_grid_search.train(x= predictors,y= target, training_frame=train)\nGLM_grid_search = h2o.load_model("/content/drive/MyDrive/Ensemble model/Heterogeneous_Ensemble_to_Predict_Credit_Card_Default_using_H2O/models/GLM_grid_model_10")\nGLM_grid_search_performance = GLM_grid_search.model_performance(test)\n '